## Data Loading

In [32]:
# import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Load Data

In [33]:
# Local authority traffic figures give the total volume (vehicle miles) of traffic across each local authority for the whole year.
pd_local_points = pd.read_csv('../data/uk_data/local_authority_traffic.csv')

In [59]:
pd_local_points.columns

Index(['local_authority_id', 'local_authority_name', 'local_authority_code',
       'year', 'link_length_km', 'link_length_miles', 'cars_and_taxis',
       'all_motor_vehicles'],
      dtype='object')

In [34]:
# Vehicle counts recorded on major and minor roads
pd_raw_count = pd.read_csv('../data/uk_data/dft_traffic_counts_raw_counts.csv')

/var/folders/_z/nz5spzqj2rl11n5lc_k1cwwc0000gn/T/ipykernel_78483/2200544698.py:2: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  pd_raw_count = pd.read_csv('../data/uk_data/dft_traffic_counts_raw_counts.csv')


In [58]:
pd_raw_count.columns

Index(['count_point_id', 'direction_of_travel', 'year', 'count_date', 'hour',
       'region_id', 'region_name', 'region_ons_code', 'local_authority_id',
       'local_authority_name', 'local_authority_code', 'road_name',
       'road_category', 'road_type', 'start_junction_road_name',
       'end_junction_road_name', 'easting', 'northing', 'latitude',
       'longitude', 'link_length_km', 'link_length_miles', 'pedal_cycles',
       'two_wheeled_motor_vehicles', 'cars_and_taxis', 'buses_and_coaches',
       'LGVs', 'HGVs_2_rigid_axle', 'HGVs_3_rigid_axle',
       'HGVs_4_or_more_rigid_axle', 'HGVs_3_or_4_articulated_axle',
       'HGVs_5_articulated_axle', 'HGVs_6_articulated_axle', 'all_HGVs',
       'all_motor_vehicles', 'Month', 'Day_of_Week', 'Day'],
      dtype='object')

## Initial Data Inspection

#### Basic Inspection of local autorities data

In [35]:
pd_local_points.head()

,local_authority_id,local_authority_name,local_authority_code,year,link_length_km,link_length_miles,cars_and_taxis,all_motor_vehicles
0,1,Isles of Scilly,E06000053,1993,36.01,22.38,9.000000e+05,1.400000e+06
1,2,Nottinghamshire,E10000024,1993,4650.69,2889.80,2.702600e+09,3.368900e+09
2,3,Glasgow City,S12000049,1993,1768.54,1098.92,1.381200e+09,1.662600e+09
3,4,North Lanarkshire,S12000050,1993,1606.71,998.36,1.253500e+09,1.547600e+09
4,5,Somerset,E06000066,1993,6630.33,4119.90,2.410300e+09,2.958200e+09


In [36]:
print(f"The shape is:\n{pd_local_points.shape}")

The shape is:
(6353, 8)


#### Basic Inspection of raw_count dataset

In [37]:
pd_raw_count.head()

,count_point_id,direction_of_travel,year,count_date,hour,region_id,region_name,region_ons_code,local_authority_id,local_authority_name,...,buses_and_coaches,LGVs,HGVs_2_rigid_axle,HGVs_3_rigid_axle,HGVs_4_or_more_rigid_axle,HGVs_3_or_4_articulated_axle,HGVs_5_articulated_axle,HGVs_6_articulated_axle,all_HGVs,all_motor_vehicles
0,51,N,2004,2004-05-21,7,1,South West,E12000009,1,Isles of Scilly,...,1.0,13,0.0,0.0,0.0,0.0,0,0.0,0.0,19.0
1,51,N,2004,2004-05-21,8,1,South West,E12000009,1,Isles of Scilly,...,2.0,13,7.0,7.0,0.0,0.0,0,0.0,14.0,44.0
2,51,N,2004,2004-05-21,9,1,South West,E12000009,1,Isles of Scilly,...,5.0,17,5.0,0.0,0.0,0.0,0,0.0,5.0,45.0
3,51,N,2004,2004-05-21,10,1,South West,E12000009,1,Isles of Scilly,...,4.0,4,4.0,0.0,0.0,0.0,0,0.0,4.0,41.0
4,51,N,2004,2004-05-21,11,1,South West,E12000009,1,Isles of Scilly,...,3.0,14,6.0,0.0,0.0,0.0,0,0.0,6.0,41.0


In [38]:
print(f"The shape is :\n{pd_raw_count.shape}")

The shape is :
(4960860, 35)


In [39]:
print(pd_local_points.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6353 entries, 0 to 6352
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   local_authority_id    6353 non-null   int64  
 1   local_authority_name  6353 non-null   object 
 2   local_authority_code  6353 non-null   object 
 3   year                  6353 non-null   int64  
 4   link_length_km        6353 non-null   float64
 5   link_length_miles     6353 non-null   float64
 6   cars_and_taxis        6353 non-null   float64
 7   all_motor_vehicles    6353 non-null   float64
dtypes: float64(4), int64(2), object(2)
memory usage: 397.2+ KB
None


In [40]:
print(pd_raw_count.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960860 entries, 0 to 4960859
Data columns (total 35 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   count_point_id                int64  
 1   direction_of_travel           object 
 2   year                          int64  
 3   count_date                    object 
 4   hour                          int64  
 5   region_id                     int64  
 6   region_name                   object 
 7   region_ons_code               object 
 8   local_authority_id            int64  
 9   local_authority_name          object 
 10  local_authority_code          object 
 11  road_name                     object 
 12  road_category                 object 
 13  road_type                     object 
 14  start_junction_road_name      object 
 15  end_junction_road_name        object 
 16  easting                       int64  
 17  northing                      int64  
 18  latitude              

## Data Quality checks

In [41]:
print("\nUnique local_authority_id in pd_local_points:", pd_local_points['local_authority_id'].nunique())
print("Total rows in pd_count_pd_local_pointspoints:", len(pd_local_points))


Unique local_authority_id in pd_local_points: 214
Total rows in pd_count_pd_local_pointspoints: 6353


### Data Formatting

In [42]:
# Check data types
print("\nData types in pd_local_points:")
print(pd_local_points.dtypes)


Data types in pd_local_points:
local_authority_id        int64
local_authority_name     object
local_authority_code     object
year                      int64
link_length_km          float64
link_length_miles       float64
cars_and_taxis          float64
all_motor_vehicles      float64
dtype: object


In [43]:
# conver `count_date` to datetime type
pd_raw_count['count_date'] =  pd.to_datetime(pd_raw_count['count_date'])

In [44]:
# Check data types
print("\nData types in pd_raw_count:")
print(pd_raw_count.dtypes)


Data types in pd_raw_count:
count_point_id                           int64
direction_of_travel                     object
year                                     int64
count_date                      datetime64[ns]
hour                                     int64
region_id                                int64
region_name                             object
region_ons_code                         object
local_authority_id                       int64
local_authority_name                    object
local_authority_code                    object
road_name                               object
road_category                           object
road_type                               object
start_junction_road_name                object
end_junction_road_name                  object
easting                                  int64
northing                                 int64
latitude                               float64
longitude                              float64
link_length_km                 

In [45]:
# Extract Date Components 
pd_raw_count['Month'] = pd_raw_count['count_date'].dt.month
pd_raw_count['Day_of_Week'] = pd_raw_count['count_date'].dt.day_name()
pd_raw_count['Day'] = pd_raw_count['count_date'].dt.day

### Check Data Duplicates

**Notes**:

- No duplicates found in either dataset as data already aggrated

In [46]:
pd_local_points.duplicated().value_counts()

False    6353
Name: count, dtype: int64

In [47]:
pd_raw_count.duplicated().value_counts()

False    4960860
Name: count, dtype: int64

### Check for Nulls

In [48]:
# Checking for missing values 
print("Missing values in pd_local_points:")
print((pd_local_points.isna().sum() * 100 / pd_local_points.shape[0]).round(2))

Missing values in pd_local_points:
local_authority_id      0.0
local_authority_name    0.0
local_authority_code    0.0
year                    0.0
link_length_km          0.0
link_length_miles       0.0
cars_and_taxis          0.0
all_motor_vehicles      0.0
dtype: float64


In [49]:
print("\nMissing values in pd_raw_count:")
print((pd_raw_count.isna().sum() * 100 / pd_raw_count.shape[0]).round(2))


Missing values in pd_raw_count:
count_point_id                   0.00
direction_of_travel              0.00
year                             0.00
count_date                       0.00
hour                             0.00
region_id                        0.00
region_name                      0.00
region_ons_code                  0.00
local_authority_id               0.00
local_authority_name             0.00
local_authority_code             0.00
road_name                        0.00
road_category                    0.00
road_type                        0.00
start_junction_road_name        54.77
end_junction_road_name          54.77
easting                          0.00
northing                         0.00
latitude                         0.00
longitude                        0.00
link_length_km                  54.72
link_length_miles               54.72
pedal_cycles                     0.00
two_wheeled_motor_vehicles       0.00
cars_and_taxis                   0.00
buses_and_coaches

**Missing Values Options**

- Impute
- Drop any rows that are affected
- Drop columns that are affected

#### Dealing Nulls

**Notes** 

Looking at the missing values in start_junction_road_name 59.87 and end_junction_road_name 59.87, we can conclude that base on our project objective and the dataset we have we can either:

- Drop the rows with the missing values 
- Remove Rows with Missing Values: If start_junction_road_name and end_junction_road_name are essential, we can consider removing rows where both are missing, especially if it doesn’t significantly reduce the dataset.
- Fill the missing values with the `road_name`, `latitude` or `longitude` values of the nearest road/count_point

In [50]:
# Remove rows where both 'start_junction_road_name' and 'end_junction_road_name' are missing
pd_raw_count_nulls = pd_raw_count.drop(columns=['start_junction_road_name', 'end_junction_road_name', 'link_length_km', 'link_length_miles'])


In [51]:
[pd_raw_count_nulls.isna().sum() > 0]

[count_point_id                  False
 direction_of_travel             False
 year                            False
 count_date                      False
 hour                            False
 region_id                       False
 region_name                     False
 region_ons_code                 False
 local_authority_id              False
 local_authority_name            False
 local_authority_code            False
 road_name                       False
 road_category                   False
 road_type                       False
 easting                         False
 northing                        False
 latitude                        False
 longitude                       False
 pedal_cycles                    False
 two_wheeled_motor_vehicles      False
 cars_and_taxis                   True
 buses_and_coaches                True
 LGVs                            False
 HGVs_2_rigid_axle                True
 HGVs_3_rigid_axle                True
 HGVs_4_or_more_rigid_axl

In [52]:
print((pd_raw_count_nulls.isna().sum() * 100 / pd_raw_count.shape[0]).round(2))

count_point_id                  0.0
direction_of_travel             0.0
year                            0.0
count_date                      0.0
hour                            0.0
region_id                       0.0
region_name                     0.0
region_ons_code                 0.0
local_authority_id              0.0
local_authority_name            0.0
local_authority_code            0.0
road_name                       0.0
road_category                   0.0
road_type                       0.0
easting                         0.0
northing                        0.0
latitude                        0.0
longitude                       0.0
pedal_cycles                    0.0
two_wheeled_motor_vehicles      0.0
cars_and_taxis                  0.0
buses_and_coaches               0.0
LGVs                            0.0
HGVs_2_rigid_axle               0.0
HGVs_3_rigid_axle               0.0
HGVs_4_or_more_rigid_axle       0.0
HGVs_3_or_4_articulated_axle    0.0
HGVs_5_articulated_axle     

#### Dealing With Nulls 

In [53]:
pd_raw_count_nulls.head()

,count_point_id,direction_of_travel,year,count_date,hour,region_id,region_name,region_ons_code,local_authority_id,local_authority_name,...,HGVs_3_rigid_axle,HGVs_4_or_more_rigid_axle,HGVs_3_or_4_articulated_axle,HGVs_5_articulated_axle,HGVs_6_articulated_axle,all_HGVs,all_motor_vehicles,Month,Day_of_Week,Day
0,51,N,2004,2004-05-21,7,1,South West,E12000009,1,Isles of Scilly,...,0.0,0.0,0.0,0,0.0,0.0,19.0,5,Friday,21
1,51,N,2004,2004-05-21,8,1,South West,E12000009,1,Isles of Scilly,...,7.0,0.0,0.0,0,0.0,14.0,44.0,5,Friday,21
2,51,N,2004,2004-05-21,9,1,South West,E12000009,1,Isles of Scilly,...,0.0,0.0,0.0,0,0.0,5.0,45.0,5,Friday,21
3,51,N,2004,2004-05-21,10,1,South West,E12000009,1,Isles of Scilly,...,0.0,0.0,0.0,0,0.0,4.0,41.0,5,Friday,21
4,51,N,2004,2004-05-21,11,1,South West,E12000009,1,Isles of Scilly,...,0.0,0.0,0.0,0,0.0,6.0,41.0,5,Friday,21


In [54]:
pd_raw_count_nulls.isna().sum()

count_point_id                   0
direction_of_travel              0
year                             0
count_date                       0
hour                             0
region_id                        0
region_name                      0
region_ons_code                  0
local_authority_id               0
local_authority_name             0
local_authority_code             0
road_name                        0
road_category                    0
road_type                        0
easting                          0
northing                         0
latitude                         0
longitude                        0
pedal_cycles                     0
two_wheeled_motor_vehicles       0
cars_and_taxis                   1
buses_and_coaches                5
LGVs                             0
HGVs_2_rigid_axle                4
HGVs_3_rigid_axle                1
HGVs_4_or_more_rigid_axle        2
HGVs_3_or_4_articulated_axle     1
HGVs_5_articulated_axle          0
HGVs_6_articulated_a

In [55]:
pd_raw_count_nulls.isnull().sum()

count_point_id                   0
direction_of_travel              0
year                             0
count_date                       0
hour                             0
region_id                        0
region_name                      0
region_ons_code                  0
local_authority_id               0
local_authority_name             0
local_authority_code             0
road_name                        0
road_category                    0
road_type                        0
easting                          0
northing                         0
latitude                         0
longitude                        0
pedal_cycles                     0
two_wheeled_motor_vehicles       0
cars_and_taxis                   1
buses_and_coaches                5
LGVs                             0
HGVs_2_rigid_axle                4
HGVs_3_rigid_axle                1
HGVs_4_or_more_rigid_axle        2
HGVs_3_or_4_articulated_axle     1
HGVs_5_articulated_axle          0
HGVs_6_articulated_a

#### Save cleaned data to a new CSV

In [56]:
pd_raw_count_nulls.shape

(4960860, 34)

In [57]:
pd_local_points.shape

(6353, 8)

In [31]:
pd_raw_count_nulls.to_csv('../data/pd_raw_count_cleaned.csv', index=False)

In [32]:
pd_local_points.to_csv('../data/pd_count_points_cleaned.csv', index=False)